In [211]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Importing Libraries

In [212]:
import pandas as pd
import numpy as np
import plotly.express as px

## Seeing the Data

In [213]:
train=pd.read_csv('/kaggle/input/titanic/train.csv')
test=pd.read_csv('/kaggle/input/titanic/test.csv')
id=test['PassengerId']
# train
test

## Info abt the Data

In [214]:
train.info()

## Describe Data

In [215]:
train.describe()

## Unique values in Data

In [216]:
train.nunique()

## Occurance of Each Values

In [217]:
train.value_counts()

## Null values in Data

In [218]:
train.isnull().sum()

## Data Analysis

In [219]:
fig=px.bar(train,x='Parch', color='Pclass',y='Sex',hover_data=['Survived'])
fig.show()

In [220]:
fig=px.bar(train,x='Parch', color='Pclass',y='Sex',hover_data=['Survived'])
fig.show()

In [221]:
fig=px.bar(train,x='Age', color='Pclass',y='Sex',hover_data=['Survived'])
fig.show()

In [222]:
fig=px.bar(train,x='Age', color='SibSp',y='Sex',hover_data=['Survived'])
fig.show()

In [223]:
fig=px.bar(train,x='Embarked', color='Pclass',y='Sex',hover_data=['Survived'])
fig.show()

In [224]:
fig=px.bar(train,x='SibSp', color='Pclass',y='Sex',hover_data=['Survived'])
fig.show()

In [225]:
fig=px.histogram(train,x='Age',color='Sex')  # Plotting Age & Sex
fig.show()

In [226]:
fig=px.histogram(train,x='Age',color='Survived')   # Whether Young people survived more
fig.show()

In [227]:
fig=px.histogram(train,x='Fare',color='Sex')  # Whether any ineuqality in Fare existed
fig.show()

In [228]:
fig=px.histogram(train,x='Fare',color='Survived')   # Wether higher fare influenced more survivors
fig.show()

## Dropping Columns

In [229]:
train=train.drop(columns=['PassengerId','Name'], axis=1)
test=test.drop(columns=['PassengerId','Name'], axis=1)
train

## Col with Null Values

In [230]:
null_train=train.columns[train.isnull().any()]
null_train

In [231]:
null_test=test.columns[test.isnull().any()]
null_test

## Seperate Cols w.r.t Datatypes

In [232]:
num_train = train.select_dtypes(include=['int64','float64','UInt32'])
cat_train = train.select_dtypes(include=['object','string'])
num_train

In [233]:
num_test = test.select_dtypes(include=['int64','float64','UInt32'])
cat_test = test.select_dtypes(include=['object','string'])
num_test

## Encoding

In [234]:
# Label Encoding - This transformer should be used to encode 1 COLUMN at a Time
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

for cat in cat_train:
    train[cat]=le.fit_transform(train[cat])
    
train

In [235]:
for cat in cat_test:
    test[cat]=le.fit_transform(test[cat])
test

## Seperate Cols having:-
* Continuous values
* Discreet values

In [236]:
con_train =[col for col in num_train if train[col].nunique()>25]
dis_train =[col for col in num_train if train[col].nunique()<25]
# dis_train
con_train

In [237]:
con_test =[col for col in num_test if test[col].nunique()>25]
dis_test =[col for col in num_test if test[col].nunique()<25]
dis_test
# con_test

## Impute Missing Values

In [238]:
from sklearn.impute import SimpleImputer

imp = SimpleImputer(strategy='most_frequent')

train[null_train] = imp.fit_transform(train[null_train])

train.isnull().sum().sum()

In [239]:
test[null_test] = imp.fit_transform(test[null_test])

test.isnull().sum().sum()

## Log Transformation - Continuous Data

In [240]:
train[con_train]=np.log1p(train[con_train])
train

In [241]:
test[con_test]=np.log1p(test[con_test])
test

## Standardizing - Discrete Values

In [242]:
# RobustScaler
from sklearn.preprocessing import RobustScaler

rs = RobustScaler()

train[dis_train]= rs.fit_transform(train[dis_train])
train

In [243]:
test[dis_test]= rs.fit_transform(test[dis_test])
test

## Removing Outliers

In [244]:
# Using Isolation Forest
from sklearn.ensemble import IsolationForest
iso = IsolationForest(contamination=0.3)

out = iso.fit_predict(train)

# select all rows that are not outliers
train[out != -1]
train

In [245]:
out = iso.fit_predict(test)

# select all rows that are not outliers
test[out != -1]
test

## Splitting X & y

In [246]:
y=train['Survived']
X=train.drop(['Survived'],axis=1)
# y
X

## Train Test Split

In [247]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)
# X_train
# y_train
# X_test
y_test

## Train

In [248]:
from xgboost import XGBClassifier

model = XGBClassifier(objective='binary:logistic', learning_rate=0.8,n_estimators = 100, max_depth=25)

model.fit(X_train,y_train)

In [249]:
predict= model.predict(X_test)
predict

## Scoring - Confusion Matrix

In [250]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test,predict)

from sklearn.metrics import ConfusionMatrixDisplay
disp = ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=model.classes_)
disp.plot()

## Scoring - F1 Score

In [251]:
from sklearn.metrics import f1_score
f1_score(y_test, predict)

## Suggestions:-
* Kaggle - https://www.kaggle.com/pythonkumar
* GitHub - https://github.com/KumarPython​
* Twitter - https://twitter.com/KumarPython
* LinkedIn - https://www.linkedin.com/in/kumarpython/

## Submission

In [252]:
submission=pd.DataFrame({'PassengerId': id,
                         'Survived' : model.predict(test)
                        })
submission

# submission.to_csv('submission.csv', index=False)